In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/cms/processed/metadata')
db_path=os.path.join(root_dir,'db/cms.db')

In [3]:
#define input tables and metadata
tbl_names_lst=['s1_ben_sum_2008','s1_ben_sum_2009','s1_ben_sum_2010','s1_carrier_1a','s1_carrier_1b','s1_inpatient','s1_outpatient','s1_prescrp']

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
    'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
    'JOIN_CNDTN':{'WHERE':0.5, 'AND':0.5} #Use WHERE or AND with JOIN CLAUSE
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

smk2


In [10]:
# Generate Single queries 
rnd_query=self.make_single_agg_query()


['s1_carrier_1a', 's1_inpatient', 's1_outpatient', 's1_prescrp', 's1_carrier_1b'] 1


In [11]:
rnd_query

{'query':       HCPCS_CD_8 HCPCS_CD_13 SP_OSTEOPRS BENE_HI_CVRAGE_TOT_MONS  \
 0           None        None           1                       0   
 1           None        None           1                       0   
 2           None        None           1                       0   
 3           None        None           1                       0   
 4           None        None           1                       0   
 ...          ...         ...         ...                     ...   
 99702      P9604        None           2                      12   
 99703      P9604       85610           2                      12   
 99704      Q0091        None           2                      12   
 99705      Q0092        None           2                      12   
 99706      R0075       74000           2                      12   
 
       LINE_PRCSG_IND_CD_9 HCPCS_CD_12 BENE_RACE_CD BENE_SMI_CVRAGE_TOT_MONS  \
 0                    None        None            1                       12   
 